# Gradient Checking for DoMINO Sensitivity Analysis

In this notebook, we will:

1. Run a baseline DoMINO inference to compute aerodynamic force and sensitivity fields.
2. Perturb the geometry along the sensitivity direction with various $\epsilon$ magnitudes to approximate drag gradients via finite differences.
3. Compare the adjoint-based gradient to the finite-difference estimates using a log-symmetrical plot.


In [23]:
### Imports
from pathlib import Path

import hydra
import numpy as np
import pyvista as pv
import torch
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from physicsnemo.distributed import DistributedManager

# Local import: the DoMINO inference pipeline defined next to this notebook
from main import DoMINOInference

# Limit GPU memory usage if on CUDA-enabled machine (optional)
if torch.cuda.is_available():
    torch.cuda.set_per_process_memory_fraction(0.9)


## Initialize Configuration and Distributed Environment
We load the Hydra configuration used during training and set up the distributed runtime (single-GPU by default).


In [24]:
### Load Hydra configuration
with hydra.initialize(version_base="1.3", config_path="conf"):
    cfg = hydra.compose(config_name="config")

### Initialise (possibly dummy) distributed context
DistributedManager.initialize()
dist = DistributedManager()

if dist.world_size > 1:
    # Ensure all ranks reach this point before continuing
    torch.distributed.barrier()

### Construct DoMINO inference pipeline
domino = DoMINOInference(
    cfg=cfg,
    model_checkpoint_path=(Path.cwd() / "DoMINO.0.41.pt"),
    dist=dist,
)


RuntimeError: Error(s) in loading state_dict for DoMINO:
	Missing key(s) in state_dict: "geo_rep_volume.geo_processors.0.device_buffer", "geo_rep_volume.geo_processors.0.encoder.layers.0.conv3d.weight", "geo_rep_volume.geo_processors.0.encoder.layers.0.conv3d.bias", "geo_rep_volume.geo_processors.0.encoder.layers.0.norm.weight", "geo_rep_volume.geo_processors.0.encoder.layers.0.norm.bias", "geo_rep_volume.geo_processors.0.encoder.layers.1.conv3d.weight", "geo_rep_volume.geo_processors.0.encoder.layers.1.conv3d.bias", "geo_rep_volume.geo_processors.0.encoder.layers.1.norm.weight", "geo_rep_volume.geo_processors.0.encoder.layers.1.norm.bias", "geo_rep_volume.geo_processors.0.encoder.layers.3.conv3d.weight", "geo_rep_volume.geo_processors.0.encoder.layers.3.conv3d.bias", "geo_rep_volume.geo_processors.0.encoder.layers.3.norm.weight", "geo_rep_volume.geo_processors.0.encoder.layers.3.norm.bias", "geo_rep_volume.geo_processors.0.encoder.layers.4.conv3d.weight", "geo_rep_volume.geo_processor

## Baseline Inference and Sensitivity Computation
We first evaluate the DoMINO model on the reference geometry to obtain aerodynamic forces and the adjoint-based sensitivity field. These sensitivities tell us how to move the surface to *decrease* drag.


In [ ]:
### Load STL geometry
input_file = Path.cwd() / "geometries" / "drivaer_1_single_solid.stl"
assert input_file.exists(), f"{input_file=} not found."

mesh: pv.PolyData = pv.read(input_file)

### Run baseline inference
results = domino(
    mesh=mesh,
    stream_velocity=38.889,  # [m/s]
    stencil_size=7,          # neighbour stencil used during training
    air_density=1.205,       # [kg/m³]
    verbose=False,
)

print(f'Baseline drag force: {results["aerodynamic_force"][0]:.3f} N')

# Attach raw results to the mesh for downstream processing (normals/projections)
for key, value in results.items():
    if len(value) == mesh.n_cells:
        mesh.cell_data[key] = value
    elif len(value) == mesh.n_points:
        mesh.point_data[key] = value

### Post-process sensitivities (projection onto normals + Laplacian smoothing)
sensitivity_results = domino.postprocess_point_sensitivities(results=results, mesh=mesh)
for key, value in sensitivity_results.items():
    mesh[key] = value


AssertionError: input_file=PosixPath('/home/psharpe/GitHub/physicsnemo-cfd/workflows/domino_design_sensitivities/geometries/drivaer_1_single_solid.stl') not found.

In [ ]:
def get_drag(epsilon: float, direction: np.ndarray) -> float:
    """Evaluate drag for geometry displaced by `epsilon * direction`.
    
    Args:
        epsilon: Perturbation magnitude.
        direction: (n_points, 3) array describing displacement direction for each point.
    Returns:
        Drag force [N] for the perturbed geometry.
    """
    # Create a warped mesh by translating each point along the supplied direction
    warped_mesh = pv.PolyData(mesh.points + epsilon * direction, mesh.faces)
    warped_results = domino(
        mesh=warped_mesh,
        stream_velocity=38.889,
        stencil_size=7,
        air_density=1.205,
        verbose=False,
    )
    return float(warped_results["aerodynamic_force"][0])


In [ ]:
### Prepare epsilon sweep
epsilons = [0.0] + [
    sign * number * 10**exponent
    for exponent in range(-16, -12)
    for number in np.logspace(0, 1, 6)
    for sign in (1, -1)
]

results_dir = Path.cwd() / "gradient_checking_results"
results_dir.mkdir(exist_ok=True)

# Helper to (re)write result files so that multiple executions overwrite previous data
def write_results(filename: Path, epsilons: list[float], drag_values: list[float]):
    data = np.column_stack([epsilons, drag_values])
    np.savetxt(filename, data, delimiter=",")

# RAW sensitivities
raw_direction = mesh.point_data["raw_sensitivity_cells"]
raw_drags: list[float] = []
for eps in epsilons:
    raw_drags.append(get_drag(eps, raw_direction))
write_results(results_dir / "drag_gradients_raw.txt", epsilons, raw_drags)

# SMOOTH sensitivities
smooth_direction = mesh.point_data["smooth_sensitivity_point"]
smooth_drags: list[float] = []
for eps in epsilons:
    smooth_drags.append(get_drag(eps, smooth_direction))
write_results(results_dir / "drag_gradients_smooth.txt", epsilons, smooth_drags)


In [ ]:
plt.figure(figsize=(9, 7))

files = {
    'Raw Sensitivities': results_dir / 'drag_gradients_raw.txt',
    'Smooth Sensitivities': results_dir / 'drag_gradients_smooth.txt',
}

for label, file in files.items():
    data = np.loadtxt(file, delimiter=",")
    eps = data[:, 0]
    drag = data[:, 1]
    baseline = drag[eps == 0][0]
    drag_delta = drag - baseline
    plt.plot(eps, drag_delta, '.', label=label)

# Analytical gradient from adjoint
analytical_grad = -376531.0
# Build a symmetric x-axis grid so the line appears continuous across 0
x_unique = np.unique(np.concatenate([line.get_xdata() for line in plt.gca().get_lines()]))
x_min = 1e-15
x_max = np.max(np.abs(x_unique[x_unique != 0]))
x_space = np.logspace(np.log10(x_min) - 3, np.log10(x_max), 100)
x_plot = np.concatenate((-x_space[::-1], x_space))
plt.plot(x_plot, analytical_grad * x_plot, '--k', label='Adjoint-Based Gradient', zorder=2)

# Symlog axes with linear region around zero
plt.xscale('symlog', linthresh=x_min)
plt.yscale('symlog', linthresh=np.abs(analytical_grad) * x_min)

# Custom symmetrical log tickers for nicer minor grids
for axis, linth in ((plt.gca().xaxis, x_min), (plt.gca().yaxis, np.abs(analytical_grad) * x_min)):
    for loc_kind, base in (('major', 1000), ('minor', 10)):
        locator = ticker.SymmetricalLogLocator(base=base, linthresh=linth)
        locator.numticks = 1000
        if loc_kind == 'major':
            axis.set_major_locator(locator)
        else:
            axis.set_minor_locator(locator)

plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.axhline(0, color='k', linewidth=1.0, alpha=0.3)
plt.axvline(0, color='k', linewidth=1.0, alpha=0.3)
plt.xlabel('Epsilon [m / (N/m)]')
plt.ylabel('Δ Drag Force relative to baseline [N]')
plt.title('Adjoint-Predicted Gradient vs Finite-Difference Estimates')
plt.legend(loc='lower left')
plt.show()
